In [4]:

# Identify 1000 users and download timelines
import json
import time

from tweepy import OAuthHandler
from tweepy.api import API
from tweepy.parsers import JSONParser

import sys
if not('/high_spd_work/sm_w2v' in sys.path):
    sys.path.append('/high_spd_work/sm_w2v')

from sm_w2v.tokens import CONSUMER_KEY, CONSUMER_SECRET, TOKEN_KEY, TOKEN_SECRET

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(TOKEN_KEY, TOKEN_SECRET)
api = API(auth, parser=JSONParser())

# this is going to download 1 timeline every 16 minutes
# about 10 days for 1000 timelines

user_str_ids = []
with open('../data/top_users_to_PrEP.txt') as f_in:
    for line_no, line in enumerate(f_in):
        user_str_ids.append(line)
        if line_no == 1000:
            break

users = []
pages = list(range(0, 150))
with open('../data/user_timeline_tweets.json', 'w') as f_out:
    for user_id in user_str_ids:
        try:
            for page in pages:
                for twt in api.user_timeline(user_id, count=20, page=page):
                    f_out.write(json.dumps(twt) + '\n')
            users.append(user_id)
            time.sleep(60 * 16)
        except:
            pass

print(users)

[]


In [2]:
# LDA topic modeling on user-timelines (3000 tweets)

In [3]:
# idenitify topics of interest

In [4]:
# show users vs. features (topics)

In [5]:
# do time-plot of features for a specific user (topics) vs. time

In [ ]:
# do Doc2Vec to cluster user-timelines (documents)